# Simulate the striatum example network

This example continues ```striatum_example.ipynb``` and shows how to setup input and run the simulation.

## Setup the input

Here we use the file ```input_config/striatum-test-input.json```.

The input can be generated in serial or in parallel. To generate it in parallel, pass ```rc``` parameter as in the previous examples.

In [1]:
import os
from snudda.input import SnuddaInput

network_path = os.path.join("networks", "striatum_example")

si = SnuddaInput(network_path=network_path, 
                 input_config_file=os.path.join("input_config","striatum-test-input.json"),
                 verbose=False)
si.generate()

Loading 20 neurons with 430 synapses and 0 gap junctions
Writing spikes to networks/striatum_example/input-spikes.hdf5


## Compile the  NEURON mod files
If you run this on Windows you need to exchange / for \ in the path below.

In [2]:
!nrnivmodl ../../snudda/data/neurons/mechanisms

/home/HDD-drive/Repos/Snudda/examples/notebooks
Mod files: "../../snudda/data/neurons/mechanisms/bk_ch.mod" "../../snudda/data/neurons/mechanisms/bk_fs.mod" "../../snudda/data/neurons/mechanisms/bk_ms.mod" "../../snudda/data/neurons/mechanisms/ca_ch.mod" "../../snudda/data/neurons/mechanisms/cadyn_fs.mod" "../../snudda/data/neurons/mechanisms/cadyn_ms.mod" "../../snudda/data/neurons/mechanisms/cal12_ms.mod" "../../snudda/data/neurons/mechanisms/cal13_ms.mod" "../../snudda/data/neurons/mechanisms/cal_ch.mod" "../../snudda/data/neurons/mechanisms/caldyn_ms.mod" "../../snudda/data/neurons/mechanisms/can_fs.mod" "../../snudda/data/neurons/mechanisms/can_ms.mod" "../../snudda/data/neurons/mechanisms/cap_ch.mod" "../../snudda/data/neurons/mechanisms/caq_fs.mod" "../../snudda/data/neurons/mechanisms/caq_ms.mod" "../../snudda/data/neurons/mechanisms/car_fs.mod" "../../snudda/data/neurons/mechanisms/car_ms.mod" "../../snudda/data/neurons/mechanisms/cat32_ms.mod" "../../snudda/data/neurons/mecha

 -> NMODL ../../../snudda/data/neurons/mechanisms/kas_fs.mod
(cd "../../../snudda/data/neurons/mechanisms"; MODLUNIT=/home/robert/.local/lib/python3.8/site-packages/neuron/.data/share/nrn/lib/nrnunits.lib /home/robert/.local/lib/python3.8/site-packages/neuron/.data/bin/nocmodl kas_fs.mod -o "/home/HDD-drive/Repos/Snudda/examples/notebooks/x86_64")
 -> NMODL ../../../snudda/data/neurons/mechanisms/kas_ms.mod
(cd "../../../snudda/data/neurons/mechanisms"; MODLUNIT=/home/robert/.local/lib/python3.8/site-packages/neuron/.data/share/nrn/lib/nrnunits.lib /home/robert/.local/lib/python3.8/site-packages/neuron/.data/bin/nocmodl kas_ms.mod -o "/home/HDD-drive/Repos/Snudda/examples/notebooks/x86_64")
 -> NMODL ../../../snudda/data/neurons/mechanisms/kcnq_ch.mod
(cd "../../../snudda/data/neurons/mechanisms"; MODLUNIT=/home/robert/.local/lib/python3.8/site-packages/neuron/.data/share/nrn/lib/nrnunits.lib /home/robert/.local/lib/python3.8/site-packages/neuron/.data/bin/nocmodl kcnq_ch.mod -o "/home

 -> Compiling cadyn_fs.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c cadyn_fs.c -o cadyn_fs.o
 -> Compiling cadyn_ms.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c cadyn_ms.c -o cadyn_ms.o
 -> Compiling cal12_ms.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c cal12_ms.c -o cal12_ms.o
 -> Compiling cal13_ms.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c cal13_ms.c -o cal13_ms.o
 -> Compiling cal_ch.c
x86_64-linux-gnu-gcc -O2   -I"../

 -> Compiling kv2_ch.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c kv2_ch.c -o kv2_ch.o
 -> Compiling Kv3_ch.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c Kv3_ch.c -o Kv3_ch.o
 -> Compiling kv4_ch.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c kv4_ch.c -o kv4_ch.o
 -> Compiling na2_ch.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neurons/mechanisms" -I.   -I/home/robert/.local/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c na2_ch.c -o na2_ch.o
 -> Compiling na3n_lts.c
x86_64-linux-gnu-gcc -O2   -I"../../../snudda/data/neur

## Run the neuron simulation

Here we run the network for 3.5 seconds.

In [3]:
!mpiexec -n 4 snudda simulate networks/striatum_example --time 3.5

numprocs=4
Creating missing directory networks/striatum_example/simulation
Created directory networks/striatum_example/simulation
Using input file networks/striatum_example/input-spikes.hdf5
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64 directory (or nrnmech.dll) then you will force a recompilation of the modules.
args: Namespace(action='simulate', disable_gj=False, disable_synapses=False, exportCoreNeuron=False, input_file=None, mech_dir=None, network_file=None, neuromodulation=None, output_file=None, path='networks/striatum_example', profile=False, randomseed=None, record_all=None, record_volt=True, time=3.5, verbose=False)
Using input file networks/striatum_example/input-spikes.hdf5
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64 directory (or nrnmech.dll) then you will force a recompilation of the modules.
args: Namespace(action='simulate', disable

See ```striatum_example_plot.ipynb``` for information on plotting a spike raster.